## 流程說明
### 1.選出有內容的檔案  
沒有內容的資料都是2MB 所以只取2MB以上的檔案
原始資料:6630 >>> 有內容的:1608

轉換率 24.3%
### 2.檔案路徑分類
照檔案類型分類 (daily,epoch,sleep,bodyC,activities)
### 3.轉檔
先轉成個別的csv，在把同類型的csv合併成在一起

比較特殊的是daily & sleep & bodyC，
daily需要再分出heartrate，
sleep需要再分出sleeplevelmap，
bodyC則是因為只有一個，而且又沒正常內容，轉檔不了。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json, os
import pandas as pd
from tqdm import tqdm
from glob import glob
from pathlib import Path
from tqdm import tqdm_notebook

有17個人 每個人有65x6份資料 

In [ ]:
17*65*6

6630

### columns

In [ ]:
daily_col = ['summaryId', 'calendarDate', 'activityType', 'activeKilocalories', 'bmrKilocalories', 
      'steps', 'distanceInMeters', 'durationInSeconds', 'activeTimeInSeconds', 'startTimeInSeconds', 'startTimeOffsetInSeconds', 
      'moderateIntensityDurationInSeconds', 'vigorousIntensityDurationInSeconds', 'floorsClimbed', 
      'minHeartRateInBeatsPerMinute', 'maxHeartRateInBeatsPerMinute', 'averageHeartRateInBeatsPerMinute', 
      'restingHeartRateInBeatsPerMinute', 'timeOffsetHeartRateSamples', 'stepsGoal', 'intensityDurationGoalInSeconds', 
      'floorsClimbedGoal', 'averageStressLevel', 'maxStressLevel', 'stressDurationInSeconds', 'restStressDurationInSeconds', 'activityStressDurationInSeconds', 
      'lowStressDurationInSeconds', 'mediumStressDurationInSeconds', 'highStressDurationInSeconds', 'stressQualifier'] #齊全的欄位
daily_col.remove('timeOffsetHeartRateSamples')

In [ ]:
epoch_col = ['summaryId', 'activityType', 'activeKilocalories', 'steps', 'distanceInMeters', 'durationInSeconds', 
        'activeTimeInSeconds', 'startTimeInSeconds', 'startTimeOffsetInSeconds', 
        'met', 'intensity', 'meanMotionIntensity', 'maxMotionIntensity']

In [ ]:
act_col = ['summaryId', 'activityId', 'activityName', 'durationInSeconds', 'startTimeInSeconds', 'startTimeOffsetInSeconds', 'activityType', 
      'averageHeartRateInBeatsPerMinute', 'averageRunCadenceInStepsPerMinute', 'averageSpeedInMetersPerSecond', 'averagePaceInMinutesPerKilometer', 
      'activeKilocalories', 'deviceName', 'distanceInMeters', 
      'maxHeartRateInBeatsPerMinute', 'maxPaceInMinutesPerKilometer', 'maxRunCadenceInStepsPerMinute', 'maxSpeedInMetersPerSecond', 'steps']

In [ ]:
sleep_col = ['summaryId', 'calendarDate', 'durationInSeconds', 'startTimeInSeconds', 'startTimeOffsetInSeconds', 'unmeasurableSleepInSeconds', 
        'deepSleepDurationInSeconds', 'lightSleepDurationInSeconds', 
        'remSleepInSeconds', 'awakeDurationInSeconds', 'sleepLevelsMap', 'validation', 'timeOffsetSleepSpo2']
sleep_col.remove('sleepLevelsMap')

In [ ]:
sleep_level_col = ['deep', 'light', 'awake']

In [ ]:
stress_col = ['summaryId', 'startTimeInSeconds','startTimeOffsetInSeconds', 'durationInSeconds',
        'calendarDate', 'timeOffsetStressLevelValues','timeOffsetBodyBatteryValues']
stress_col.remove('timeOffsetStressLevelValues')
stress_col.remove('timeOffsetBodyBatteryValues')

In [ ]:
stress_col_2 = ['summaryId','startTimeInSeconds','time','timeOffsetStressLevelValues']

### stressdetails

In [ ]:
for x in tqdm(range(1,18)):
  path = '/content/drive/MyDrive/碩一下/多變量分析/data/raw data/'+str(x)+'/'

  paths = []
  files = os.listdir(path)
  for j in files:
    paths.append(path+j)

  count=0
  ok_paths=[]
  for i in paths:
    size =Path(i).stat().st_size
    if size >2:
      ok_paths.append(i)
      #print(i," size is:", size,"bytes")
      count=count+1

  daily,epoch,sleep,activities,stressdetails,=[],[],[],[],[]

  for i in ok_paths:
    ilist = i.split('/')
    kind = ilist[9][10:]
    
    if kind == 'dailies.txt':
      daily.append(i)
    elif kind == 'epochs.txt':
      epoch.append(i)
    elif kind == 'stressdetails.txt':
      stressdetails.append(i)
    elif kind == 'sleeps.txt':
      sleep.append(i)
    elif kind == 'activities.txt':
      activities.append(i)

  contents = []
  for i in stressdetails:
    with open(i,'r') as fin:
      content = json.load(fin)
      contents.append(content)

  stressdetails=[]
  for i in range(len(contents)):
    stress_list = []
    for record in range(len(contents[i])):
      temp = []
      for key in stress_col:
        try:
          temp.append(contents[i][record][key])
        except:
          temp.append(None)
      stress_list.append(temp)
    stressdetails.append(stress_list)



  for i in range(len(stressdetails)):
    df = pd.DataFrame(stressdetails[i],columns = stress_col)
    df.to_csv('/content/drive/MyDrive/碩一下/多變量分析/data/stress/'+str(i)+'.csv',index=False)

  files = glob('/content/drive/MyDrive/碩一下/多變量分析/data/stress/*.csv')
  df = pd.concat((pd.read_csv(file, usecols=stress_col)for file in files), ignore_index=True)

  df['id'] = x
  df.to_csv('/content/drive/MyDrive/碩一下/多變量分析/data/stress/id/'+str(x)+'.csv',index=False)

  for file in glob("/content/drive/MyDrive/碩一下/多變量分析/data/stress/*.csv"):
    os.remove(file)

100%|██████████| 17/17 [00:10<00:00,  1.60it/s]


### timeOffsetStressLevelValues 

In [ ]:
for x in tqdm(range(1,18)):
  path = '/content/drive/MyDrive/碩一下/多變量分析/data/raw data/'+str(x)+'/'

  paths = []
  files = os.listdir(path)
  for j in files:
    paths.append(path+j)

  count=0
  ok_paths=[]
  for i in paths:
    size =Path(i).stat().st_size
    if size >2:
      ok_paths.append(i)
      #print(i," size is:", size,"bytes")
      count=count+1

  daily,epoch,sleep,activities,stressdetails,=[],[],[],[],[]

  for i in ok_paths:
    ilist = i.split('/')
    kind = ilist[9][10:]
    
    if kind == 'stressdetails.txt':
      stressdetails.append(i)

  contents = []
  for i in stressdetails:
    with open(i,'r') as fin:
      content = json.load(fin)
      contents.append(content)

  stressdetails = []
  for i in range(len(contents)):
    stress_list = []
    for record in range(len(contents[i])):
      for key,value in zip(contents[i][record]['timeOffsetStressLevelValues'].keys(),contents[i][record]['timeOffsetStressLevelValues'].values()):
        
        try:
          stress_list.append([contents[i][record]['summaryId'],contents[i][record]['startTimeInSeconds'],key,value])
        except:
          stress_list.append(None)
            
    stressdetails.append(stress_list)

  for file in glob("/content/drive/MyDrive/碩一下/多變量分析/data/stresslevel/*.csv"):
    os.remove(file)

  for i in range(len(stressdetails)):
    df = pd.DataFrame(stressdetails[i],columns = ['summaryId','startTimeInSeconds','time','timeOffsetStressLevelValues'])
    df.to_csv('/content/drive/MyDrive/碩一下/多變量分析/data/stresslevel/'+str(i)+'.csv',index=False)

  files = glob('/content/drive/MyDrive/碩一下/多變量分析/data/stresslevel/*.csv')
  df = pd.concat((pd.read_csv(file, usecols=['summaryId','startTimeInSeconds','time','timeOffsetStressLevelValues'])for file in files), ignore_index=True)

  df['id'] = x
  df.to_csv('/content/drive/MyDrive/碩一下/多變量分析/data/stresslevel/id/'+str(x)+'.csv',index=False)

  for file in glob("/content/drive/MyDrive/碩一下/多變量分析/data/stresslevel/*.csv"):
    os.remove(file)

100%|██████████| 17/17 [00:15<00:00,  1.07it/s]


### sleep 

In [ ]:
for x in tqdm(range(1,18)):
  path = '/content/drive/MyDrive/碩一下/多變量分析/data/raw data/'+str(x)+'/'

  paths = []
  files = os.listdir(path)
  for j in files:
    paths.append(path+j)

  count=0
  ok_paths=[]
  for i in paths:
    size =Path(i).stat().st_size
    if size >2:
      ok_paths.append(i)
      #print(i," size is:", size,"bytes")
      count=count+1

  daily,epoch,sleep,activities,stressdetails,=[],[],[],[],[]

  for i in ok_paths:
    ilist = i.split('/')
    kind = ilist[9][10:]
    
    if kind == 'dailies.txt':
      daily.append(i)
    elif kind == 'epochs.txt':
      epoch.append(i)
    elif kind == 'stressdetails.txt':
      stressdetails.append(i)
    elif kind == 'sleeps.txt':
      sleep.append(i)
    elif kind == 'activities.txt':
      activities.append(i)

  contents = []
  for i in sleep:
    with open(i,'r') as fin:
      content = json.load(fin)
      contents.append(content)

  all_sleep_list=[]
  for i in range(len(contents)):
    sleep_list = []
    for record in range(len(contents[i])):
      temp = []
      for key in sleep_col:
        try:
          temp.append(contents[i][record][key])
        except:
          temp.append(None)
      sleep_list.append(temp)
    all_sleep_list.append(sleep_list)

  for file in glob("/content/drive/MyDrive/碩一下/多變量分析/data/sleep/*.csv"):
    os.remove(file)

  for i in range(len(all_sleep_list)):
    df = pd.DataFrame(all_sleep_list[i],columns = sleep_col)
    df.to_csv('/content/drive/MyDrive/碩一下/多變量分析/data/sleep/'+str(i)+'.csv',index=False)

  files = glob('/content/drive/MyDrive/碩一下/多變量分析/data/sleep/*.csv')
  df = pd.concat((pd.read_csv(file, usecols=sleep_col)for file in files), ignore_index=True)

  df['id'] = x
  df.to_csv('/content/drive/MyDrive/碩一下/多變量分析/data/sleep/id/'+str(x)+'.csv',index=False)

  for file in glob("/content/drive/MyDrive/碩一下/多變量分析/data/sleep/*.csv"):
    os.remove(file)

100%|██████████| 17/17 [00:10<00:00,  1.61it/s]


### sleepLevelsMap

In [ ]:
for x in tqdm(range(1,18)):
  path = '/content/drive/MyDrive/碩一下/多變量分析/data/raw data/'+str(x)+'/'

  paths = []
  files = os.listdir(path)
  for j in files:
    paths.append(path+j)

  count=0
  ok_paths=[]
  for i in paths:
    size =Path(i).stat().st_size
    if size >2:
      ok_paths.append(i)
      #print(i," size is:", size,"bytes")
      count=count+1

  daily,epoch,sleep,activities,stressdetails,=[],[],[],[],[]

  for i in ok_paths:
    ilist = i.split('/')
    kind = ilist[9][10:]
    
    if kind == 'sleeps.txt':
      sleep.append(i)
    

  contents = []
  for i in sleep:
    with open(i,'r') as fin:
      content = json.load(fin)
      contents.append(content)

  sleeplevelmap=[]
  for i in range(len(contents)):    
    levelmap = []
    for record in range(len(contents[i])):
      for key in contents[i][record]['sleepLevelsMap'].keys():
        for record_len in range(len(contents[i][record]['sleepLevelsMap'][key])):
          levelmap.append([contents[i][record]['summaryId'],
                  key,
                  contents[i][record]['sleepLevelsMap'][key][record_len]['startTimeInSeconds'],
                  contents[i][record]['sleepLevelsMap'][key][record_len]['endTimeInSeconds']])
        sleeplevelmap.append(levelmap)

  for file in glob("/content/drive/MyDrive/碩一下/多變量分析/data/sleeplevel/*.csv"):
    os.remove(file)

  for i in range(len(sleeplevelmap)):
    df = pd.DataFrame(sleeplevelmap[i],columns = ['summaryId','level','start','end'])
    df.to_csv('/content/drive/MyDrive/碩一下/多變量分析/data/sleeplevel/'+str(i)+'.csv',index=False)

  files = glob('/content/drive/MyDrive/碩一下/多變量分析/data/sleeplevel/*.csv')
  df = pd.concat((pd.read_csv(file, usecols=['summaryId','level','start','end'])for file in files), ignore_index=True)

  df['id'] = x
  df.to_csv('/content/drive/MyDrive/碩一下/多變量分析/data/sleeplevel/id/'+str(x)+'.csv',index=False)

  for file in glob("/content/drive/MyDrive/碩一下/多變量分析/data/sleeplevel/*.csv"):
    os.remove(file)

100%|██████████| 17/17 [01:40<00:00,  5.94s/it]


In [ ]:
sleeplevelmap=[]
for i in tqdm(range(len(contents))):    
    levelmap = []
    for record in range(len(contents[i])):
        for key in contents[i][record]['sleepLevelsMap'].keys():
            for record_len in range(len(contents[i][record]['sleepLevelsMap'][key])):
                levelmap.append([contents[i][record]['summaryId'],
                                 key,
                                 contents[i][record]['sleepLevelsMap'][key][record_len]['startTimeInSeconds'],
                                 contents[i][record]['sleepLevelsMap'][key][record_len]['endTimeInSeconds']])
            sleeplevelmap.append(levelmap)

100%|██████████| 374/374 [00:00<00:00, 7162.42it/s]


In [ ]:
for i in range(len(sleeplevelmap)):
  df = pd.DataFrame(sleeplevelmap[i],columns = ['summaryId','level','start','end'])
  df.to_csv('/content/drive/MyDrive/碩一下/多變量分析/data/sleeplevel/'+str(i)+'.csv',index=False)

files = glob('/content/drive/MyDrive/碩一下/多變量分析/data/sleeplevel/*.csv')
df = pd.concat((pd.read_csv(file, usecols=['summaryId','level','start','end'])for file in files), ignore_index=True)



In [ ]:
len(files)

6095

### daily : 有兩層

In [ ]:
for x in tqdm(range(1,18)):
  path = '/content/drive/MyDrive/碩一下/多變量分析/data/raw data/'+str(x)+'/'

  paths = []
  files = os.listdir(path)
  for j in files:
    paths.append(path+j)
  
  count=0
  ok_paths=[]
  for i in paths:
    size =Path(i).stat().st_size
    if size >2:
      ok_paths.append(i)
      #print(i," size is:", size,"bytes")
      count=count+1

  daily,epoch,sleep,activities,stressdetails,=[],[],[],[],[]
  
  for i in ok_paths:
    ilist = i.split('/')
    kind = ilist[9][10:]
    
    if kind == 'dailies.txt':
      daily.append(i)
    elif kind == 'epochs.txt':
      epoch.append(i)
    elif kind == 'stressdetails.txt':
      stressdetails.append(i)
    elif kind == 'sleeps.txt':
      sleep.append(i)
    elif kind == 'bodyComps.txt':
      bodyC.append(i)
    elif kind == 'activities.txt':
      activities.append(i)
  contents = []
  for i in daily:
    with open(i,'r') as fin:
      content = json.load(fin)
      contents.append(content)

  all_daily_list=[]
  for i in range(len(contents)):
    daily_list = []
    for record in range(len(contents[i])):
      temp = []
      for key in daily_col:
        try:
          temp.append(contents[i][record][key])
        except:
          temp.append(None)
      daily_list.append(temp)
    all_daily_list.append(daily_list)

  for file in glob("/content/drive/MyDrive/碩一下/多變量分析/data/daily/*.csv"):
    os.remove(file)

  for i in range(len(all_daily_list)):
    df = pd.DataFrame(all_daily_list[i],columns = daily_col)
    df.to_csv('/content/drive/MyDrive/碩一下/多變量分析/data/daily/'+str(i)+'.csv',index=False)

  files = glob('/content/drive/MyDrive/碩一下/多變量分析/data/daily/*.csv')
  df = pd.concat((pd.read_csv(file, usecols=daily_col)for file in files), ignore_index=True)

  df['id'] = x
  df.to_csv('/content/drive/MyDrive/碩一下/多變量分析/data/daily/id/'+str(x)+'.csv',index=False)

  for file in glob("/content/drive/MyDrive/碩一下/多變量分析/data/daily/*.csv"):
    os.remove(file)

### epoch：只有一層

In [ ]:
for x in tqdm(range(1,18)):
  path = '/content/drive/MyDrive/碩一下/多變量分析/data/raw data/'+str(x)+'/'

  paths = []
  files = os.listdir(path)
  for j in files:
    paths.append(path+j)
  
  count=0
  ok_paths=[]
  for i in paths:
    size =Path(i).stat().st_size
    if size >2:
      ok_paths.append(i)
      #print(i," size is:", size,"bytes")
      count=count+1

  daily,epoch,sleep,activities,stressdetails,=[],[],[],[],[]
  
  for i in ok_paths:
    ilist = i.split('/')
    kind = ilist[9][10:]
    
    if kind == 'epochs.txt':
      epoch.append(i)
    
  contents = []
  for i in epoch:
    with open(i,'r') as fin:
      content = json.load(fin)
      contents.append(content)

  all_epoch_list=[]
  for i in range(len(contents)):
      epoch_list = []
      for record in range(len(contents[i])):
          temp = []
          for key in epoch_col:
              try:
                  temp.append(contents[i][record][key])
              except:
                  temp.append(None)
          epoch_list.append(temp)
      all_epoch_list.append(epoch_list)

  for file in tqdm(glob("/content/drive/MyDrive/碩一下/多變量分析/data/epoch/*.csv")):
    os.remove(file)

  for i in range(len(all_epoch_list)):
    df = pd.DataFrame(all_epoch_list[i],columns = epoch_col)
    df.to_csv('/content/drive/MyDrive/碩一下/多變量分析/data/epoch/'+str(i)+'.csv',index=False)

  from glob import glob
  files = glob('/content/drive/MyDrive/碩一下/多變量分析/data/epoch/*.csv')
  df = pd.concat((pd.read_csv(file, usecols=epoch_col)for file in files), ignore_index=True)

  df['id']=x
  df.to_csv('/content/drive/MyDrive/碩一下/多變量分析/data/epoch/id/'+str(x)+'.csv',index=False)

  for file in glob("/content/drive/MyDrive/碩一下/多變量分析/data/epoch/*.csv"):
    os.remove(file)

### HeartRate

In [ ]:
for x in tqdm([1,2,3,5,6,7,8,9,10,11,12,13,14,15,16,17]):
  path = '/content/drive/MyDrive/碩一下/多變量分析/data/raw data/'+str(x)+'/'

  paths = []
  files = os.listdir(path)
  for j in files:
    paths.append(path+j)
  
  count=0
  ok_paths=[]
  for i in paths:
    size =Path(i).stat().st_size
    if size >2:
      ok_paths.append(i)
      #print(i," size is:", size,"bytes")
      count=count+1

  daily,epoch,sleep,activities,stressdetails,=[],[],[],[],[]
  
  for i in ok_paths:
    ilist = i.split('/')
    kind = ilist[9][10:]
    
    if kind == 'dailies.txt':
      daily.append(i)
    elif kind == 'epochs.txt':
      epoch.append(i)
    elif kind == 'stressdetails.txt':
      stressdetails.append(i)
    elif kind == 'sleeps.txt':
      sleep.append(i)
    elif kind == 'activities.txt':
      activities.append(i)
  contents = []
  for i in daily:
    with open(i,'r') as fin:
      content = json.load(fin)
      contents.append(content)

  daily_heartrate = []
  for i in range(len(contents)):
    daily_list = []
    for record in range(len(contents[i])):
      for key,value in zip(contents[i][record]['timeOffsetHeartRateSamples'].keys(),contents[i][record]['timeOffsetHeartRateSamples'].values()):
        
        try:
          daily_list.append([contents[i][record]['summaryId'],contents[i][record]['startTimeInSeconds'],key,value])
        except:
          daily_list.append(None)
          
  daily_heartrate.append(daily_list)

  for file in glob("/content/drive/MyDrive/碩一下/多變量分析/data/heartrate/*.csv"):
    os.remove(file)
  
  for i in range(len(daily_heartrate)):
    df = pd.DataFrame(daily_heartrate[i],columns = ['summaryId','startTimeInSeconds','TimeInSeconds','heartrate'])
    df.to_csv('/content/drive/MyDrive/碩一下/多變量分析/data/heartrate/'+str(i)+'.csv',index=False)

  files = glob('/content/drive/MyDrive/碩一下/多變量分析/data/heartrate/*.csv')
  df = pd.concat((pd.read_csv(file, usecols=['summaryId','startTimeInSeconds','TimeInSeconds','heartrate'])for file in files), ignore_index=True)

  df['id'] = x
  df.to_csv('/content/drive/MyDrive/碩一下/多變量分析/data/heartrate/id/'+str(x)+'.csv',index=False)

  for file in glob("/content/drive/MyDrive/碩一下/多變量分析/data/heartrate/*.csv"):
    os.remove(file)

100%|██████████| 16/16 [00:15<00:00,  1.02it/s]


4 有一些有問題 分開處理

In [ ]:
path = '/content/drive/MyDrive/碩一下/多變量分析/data/raw data/4/'

paths = []
files = os.listdir(path)
for j in files:
  paths.append(path+j)

count=0
ok_paths=[]
for i in paths:
  size =Path(i).stat().st_size
  if size >2:
    ok_paths.append(i)
    #print(i," size is:", size,"bytes")
    count=count+1

daily,epoch,sleep,activities,stressdetails,=[],[],[],[],[]

for i in ok_paths:
  ilist = i.split('/')
  kind = ilist[9][10:]
  
  if kind == 'dailies.txt':
    daily.append(i)

In [ ]:
contents = []
for i in daily:
  with open(i,'r') as fin:
    content = json.load(fin)
    contents.append(content)

del contents[21]

daily_heartrate = []
for i in tqdm(range(len(contents))):
  daily_list = []
  for record in range(len(contents[i])):
    for key,value in zip(contents[i][record]['timeOffsetHeartRateSamples'].keys(),contents[i][record]['timeOffsetHeartRateSamples'].values()):
      
      try:
        daily_list.append([contents[i][record]['summaryId'],contents[i][record]['startTimeInSeconds'],key,value])
      except:
        daily_list.append(None)
        
daily_heartrate.append(daily_list)

100%|██████████| 43/43 [00:00<00:00, 87.01it/s]


In [ ]:
for file in glob("/content/drive/MyDrive/碩一下/多變量分析/data/heartrate/*.csv"):
  os.remove(file)

for i in range(len(daily_heartrate)):
  df = pd.DataFrame(daily_heartrate[i],columns = ['summaryId','startTimeInSeconds','TimeInSeconds','heartrate'])
  df.to_csv('/content/drive/MyDrive/碩一下/多變量分析/data/heartrate/'+str(i)+'.csv',index=False)

files = glob('/content/drive/MyDrive/碩一下/多變量分析/data/heartrate/*.csv')
df = pd.concat((pd.read_csv(file, usecols=['summaryId','startTimeInSeconds','TimeInSeconds','heartrate'])for file in files), ignore_index=True)

df['id'] = 4
df.to_csv('/content/drive/MyDrive/碩一下/多變量分析/data/heartrate/id/4.csv',index=False)

for file in glob("/content/drive/MyDrive/碩一下/多變量分析/data/heartrate/*.csv"):
  os.remove(file)

In [ ]:
# del contents[21]

### activities 

In [ ]:
for x in tqdm([2,3,4,6,7,8,9,11,12,13,14,17]):
  path = '/content/drive/MyDrive/碩一下/多變量分析/data/raw data/'+str(x)+'/'

  paths = []
  files = os.listdir(path)
  for j in files:
    paths.append(path+j)
  
  count=0
  ok_paths=[]
  for i in paths:
    size =Path(i).stat().st_size
    if size >2:
      ok_paths.append(i)
      #print(i," size is:", size,"bytes")
      count=count+1

  daily,epoch,sleep,activities,stressdetails,=[],[],[],[],[]
  
  for i in ok_paths:
    ilist = i.split('/')
    kind = ilist[9][10:]
    
    if kind == 'activities.txt':
      activities.append(i)

  contents = []
  for i in activities:
    with open(i,'r') as fin:
      content = json.load(fin)
      contents.append(content)

  all_act_list=[]
  for i in range(len(contents)):
      act_list = []
      for record in range(len(contents[i])):
          temp = []
          for key in act_col:
              try:
                  temp.append(contents[i][record][key])
              except:
                  temp.append(None)
          act_list.append(temp)
      all_act_list.append(act_list)

  for file in glob("/content/drive/MyDrive/碩一下/多變量分析/data/act/*.csv"):
    os.remove(file)

  for i in range(len(all_act_list)):
    df = pd.DataFrame(all_act_list[i],columns = act_col)
    df.to_csv('/content/drive/MyDrive/碩一下/多變量分析/data/act/'+str(i)+'.csv',index=False)

  files = glob('/content/drive/MyDrive/碩一下/多變量分析/data/act/*.csv')
  df = pd.concat((pd.read_csv(file, usecols=act_col)for file in files), ignore_index=True)

  df['id']=x
  df.to_csv('/content/drive/MyDrive/碩一下/多變量分析/data/act/id/'+str(x)+'.csv',index=False)

  for file in glob("/content/drive/MyDrive/碩一下/多變量分析/data/act/*.csv"):
    os.remove(file)

100%|██████████| 12/12 [00:03<00:00,  3.99it/s]
